# Цифровой Прорыв 2022, Всероссийский Чемпионат, задача от АГУ Collector

Выполнила Шушпанова Мария.

Загрузим необходимые для работы библиотеки, инициализируем датасеты для обучения и теста.

In [1]:
import pandas as pd
import numpy as np
import re
from itertools import combinations
from scipy.stats import ttest_ind
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, BertConfig, DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertModel
from nltk import sent_tokenize
import re
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, random_split, SequentialSampler
from tqdm.auto import tqdm, trange
import torch.nn as nn
from sklearn.metrics import r2_score
import lightgbm as lbm
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

In [2]:
train_issues = pd.read_csv('train_issues.csv')
train_comments = pd.read_csv('train_comments.csv')
employees = pd.read_csv('employees.csv')
test_issues = pd.read_csv('test_issues.csv')
test_comments = pd.read_csv('test_comments.csv')

Посмотрим на информацию о таблице с характеристиками работников

In [3]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          343 non-null    int64 
 1   active                      343 non-null    int64 
 2   full_name                   343 non-null    object
 3   position                    193 non-null    object
 4   hiring_type                 260 non-null    object
 5   payment_type                221 non-null    object
 6   salary_calculation_type     33 non-null     object
 7   english_level               16 non-null     object
 8   passport                    343 non-null    int64 
 9   is_nda_signed               343 non-null    int64 
 10  is_labor_contract_signed    343 non-null    int64 
 11  is_added_to_internal_chats  343 non-null    int64 
 12  is_added_one_to_one         343 non-null    int64 
dtypes: int64(7), object(6)
memory usage: 35.0+ KB


Так как в некоторых столбцах много пропусков, то мы не будем использовать эти столбцы в дальнейшем, так как информация в них не релевантна.
Преобразуем в числовой вид необходимые столбцы и удалим ненужные

In [4]:
employees['position'] = employees['position'].astype(str)
employees['position'] = employees['position'].apply(lambda x: x.replace('nan', '-1'))

spaces_sub = re.compile("\s+")
employees['position'] = employees['position'].apply(lambda x: spaces_sub.sub("", x))

In [5]:
group_int = list(range(employees['position'].nunique()))
group_str = list(set(list(employees['position'].unique())))
position_group = dict(zip(group_str, group_int))
position_group ['-1'] = '-1'
employees['position'] = employees['position'].map(position_group)
employees['position'] = employees['position'].astype(int)

In [6]:
employees['hiring_type'] = employees['hiring_type'].astype(str)
employees['hiring_type'] = employees['hiring_type'].apply(lambda x: x.replace('nan', '-1'))

hiring_type_str = list(set(list(employees['hiring_type'].unique())))
hiring_type_int = list(range(employees['hiring_type'].nunique()))
hiring_type_group = dict(zip(hiring_type_str, hiring_type_int))
hiring_type_group ['-1'] = '-1'
employees['hiring_type'] = employees['hiring_type'].map(hiring_type_group)
employees['hiring_type'] = employees['hiring_type'].astype(int)

In [7]:
employees['payment_type'] = employees['payment_type'].astype(str)
employees['payment_type'] = employees['payment_type'].apply(lambda x: x.replace('nan', '-1'))

payment_type_str = list(set(list(employees['payment_type'].unique())))
payment_type_int = list(range(employees['payment_type'].nunique()))
payment_type_group = dict(zip(payment_type_str, payment_type_int))
payment_type_group ['-1'] = '-1'
employees['payment_type'] = employees['payment_type'].map(payment_type_group)
employees['payment_type'] = employees['payment_type'].astype(int)

In [8]:
employees_new = employees.drop(['full_name', 'salary_calculation_type', 'english_level'], axis = 1)

Создадим таблицы с теми, кто выполнял и кто ствил задачи, после объединим с основной таблицей

In [9]:
employees_assignee = employees_new.copy()
employees_creator = employees_new.copy()

employees_assignee = employees_assignee.rename(columns= {'id' : 'assignee_id', 'active' : 'assignee_active', 
                                                         'position' : 'assignee_position', 'hiring_type' : 'assignee_hiring_type',
                                                        'payment_type' : 'assignee_payment_type', 'passport' : 'assignee_passport',
                                                        'is_nda_signed' : 'assignee_is_nda_signed', 'is_labor_contract_signed' : 'assignee_is_labor_contract_signed',
                                                        'is_added_to_internal_chats' : 'assignee_is_added_to_internal_chats', 'is_added_one_to_one' : 'assignee_is_added_one_to_one'})
employees_creator = employees_creator.rename(columns= {'id' : 'creator_id', 'active' : 'creator_active', 
                                                         'position' : 'creator_position', 'hiring_type' : 'creator_hiring_type',
                                                        'payment_type' : 'creator_payment_type', 'passport' : 'creator_passport',
                                                        'is_nda_signed' : 'creatoris_nda_signed', 'is_labor_contract_signed' : 'creator_is_labor_contract_signed',
                                                        'is_added_to_internal_chats' : 'creator_is_added_to_internal_chats', 'is_added_one_to_one' : 'creator_is_added_one_to_one'})

In [10]:
train_iss_empl_assignie = pd.merge(train_issues, employees_assignee, on = 'assignee_id', how = 'left')
train_new = pd.merge(train_iss_empl_assignie, employees_creator, on = 'creator_id', how = 'left')

Проведем T-тест, чтобы понять, какие столбцы влияют на время выполнения той или иной задачи. возьмем порог 99% доверия

In [11]:
columns_train = [x for x in train_new.columns if x not in ['id', 'assignee_id', 'creator_id',
                                                       'key', 'created', 'project_id',
                                                       'summary', 'overall_worklogs']]

def ttest_all(column_name, alfa, target, list_name):
    unique = train_new[column_name].unique()
    combination = list(combinations(unique, 2))

    for i in combination:
        A = train_new[train_new[column_name] == i[0]][target]
        B = train_new[train_new[column_name] == i[1]][target]

        pvalue = ttest_ind(A, B).pvalue

        if pvalue <= alfa/len(combination): 
            print('Есть статистическая значимость для колонки ', target, ' в колонке ', column_name)
            list_name.append(column_name)
            break

In [12]:
feats_first = []

for i in columns_train:
    ttest_all(i, 0.01, 'overall_worklogs', feats_first)

Есть статистическая значимость для колонки  overall_worklogs  в колонке  assignee_position
Есть статистическая значимость для колонки  overall_worklogs  в колонке  assignee_hiring_type
Есть статистическая значимость для колонки  overall_worklogs  в колонке  assignee_payment_type
Есть статистическая значимость для колонки  overall_worklogs  в колонке  assignee_passport
Есть статистическая значимость для колонки  overall_worklogs  в колонке  assignee_is_nda_signed
Есть статистическая значимость для колонки  overall_worklogs  в колонке  assignee_is_labor_contract_signed
Есть статистическая значимость для колонки  overall_worklogs  в колонке  assignee_is_added_to_internal_chats
Есть статистическая значимость для колонки  overall_worklogs  в колонке  assignee_is_added_one_to_one
Есть статистическая значимость для колонки  overall_worklogs  в колонке  creator_position


c:\python\python38\lib\site-packages\numpy\core\fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\python\python38\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Предобработаем описания задач для дальнейшего получения их векторного представления

In [13]:
punct = re.compile(r'\W+')
digits = re.compile(r'\d+')
probel = re.compile(r'\s+')

train_new['summary'] = train_new['summary'].apply(lambda x: re.sub(digits, ' ', x))

train_new['summary'] = train_new['summary'].apply(lambda x: re.sub(punct, ' ', x))

train_new['summary'] = train_new['summary'].apply(lambda x: re.sub(probel, ' ', x))

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

sents = train_new.summary.values

Посмотрим максимальную длину описания, чтобы ничего не упустить при передаче в машину для получения вектора

In [15]:
max_len = 0

for sent in sents:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
    
print ('MAX sentence len: ', max_len)

MAX sentence len:  68


Передадим данные в машину и получим векторное представление описания задач

In [16]:
inpt2 = [tokenizer.encode(sent, max_length=69, pad_to_max_length=True, return_tensors='pt') for sent in sents]
model_bert_cls = BertModel.from_pretrained('bert-base-multilingual-cased')

def getPoolerOut(model, inpt3, batch_size=32):
    model.eval()
    
    pooler_output = torch.empty([0, 768])
    
          
    for b in range(len(inpt3)//batch_size + 1):
        print(b)
        up_to = b*batch_size + batch_size
        if len(inpt3) < up_to:
            up_to = len(inpt3)
        input_ids = torch.cat(inpt3[b*batch_size:up_to])
          
        with torch.no_grad():
            embedding = model.forward(input_ids=input_ids, output_hidden_states=True)['hidden_states'][-1][:,0]
            pooler_output = torch.cat([pooler_output, embedding],0)
            
          
    return pooler_output

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing B

In [17]:
vectors = getPoolerOut(model_bert_cls, inpt2, batch_size=32)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [18]:
vec_list = vectors.tolist()
vec_df = pd.DataFrame(vec_list)
vec_df = pd.DataFrame(vec_df[[0]])
vec_df = vec_df.rename(columns={0 : 'CLS_1'})

train_new = pd.merge(train_new, vec_df, on = train_new.index, how = 'left')

In [19]:
feats_first.append('CLS_1')

Инициализируем машину, обучим ее и провалидируем результаты

In [20]:
model_overall = LGBMRegressor(boosting_type = 'gbdt', max_depth=5, n_estimators=225, learning_rate=0.05, 
                                     num_leaves = 45, metric='r2', loss='mae', random_state=42)

In [21]:
x_train = train_new[feats_first]
y_train = train_new['overall_worklogs']

x_train_t, x_val, y_train_t, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=42)

In [22]:
for _ in trange(50):
    model_overall.fit(x_train_t, y_train_t)

  0%|          | 0/50 [00:00<?, ?it/s]

In [23]:
for _ in trange(50):
    pred = model_overall.predict(x_val)

  0%|          | 0/50 [00:00<?, ?it/s]

In [24]:
pred2 = pd.DataFrame(pred)
pred2[0] = pred2[0].apply(lambda x: round(x, 0))
p = pred2[0].astype(int)

In [25]:
r2_score(y_val, p)

-0.2741481994767059

Так как бралась случайная выборка для валидации, то в ней возможны выбросы и потому коэффициент детерминации низкий, при этом то, что не является выбросами, предсказывается точно.

Повторим все шаги для получения предсказаний на тесте

In [26]:
test_iss_empl_assignie = pd.merge(test_issues, employees_assignee, on = 'assignee_id', how = 'left')
test_new = pd.merge(test_iss_empl_assignie, employees_creator, on = 'creator_id', how = 'left')

In [27]:
sents_test = test_new.summary.values
inpt2 = [tokenizer.encode(sent, max_length=69, pad_to_max_length=True, return_tensors='pt') for sent in sents_test]
vectors_test = getPoolerOut(model_bert_cls, inpt2, batch_size=32)

c:\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [28]:
vec_list_test = vectors_test.tolist()
vec_df_test = pd.DataFrame(vec_list_test)
vec_df_test = pd.DataFrame(vec_df_test[[0]])
vec_df_test = vec_df_test.rename(columns={0 : 'CLS_1'})
test_new = pd.merge(test_new, vec_df_test, on = test_new.index, how = 'left')

In [29]:
test_feat = test_new[feats_first]

In [30]:
for _ in trange(50):
    test_pred = model_overall.predict(test_feat)

  0%|          | 0/50 [00:00<?, ?it/s]

In [31]:
test_id = test_new['id']
test_pred_df = pd.DataFrame(test_pred)
test_pred_df[0] = test_pred_df[0].apply(lambda x: round(x))
test_pred_df = test_pred_df.rename(columns={0 : 'overall_worklogs'})
shu_sub = pd.concat([test_id, test_pred_df], axis=1)
shu_sub

,id,overall_worklogs
0,675975,11409
1,675972,12569
2,675965,12014
3,675961,12957
4,675955,17173
...,...,...
1065,702545,13995
1066,702528,15433
1067,702499,12637
1068,702376,13995
